# Instalar pacotes e configurar ambiente

Baixar e instalar pacotes atualizados

In [ ]:
!pip install --upgrade pandas pyarrow gspread

Importar dependências

In [34]:
from google.colab import drive
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd

Conectar com o Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

Criar pastas do projeto

In [44]:
!mkdir -p "/content/drive/MyDrive/Sheets2DuckDB/Input/GoogleSheets"
!mkdir -p "/content/drive/MyDrive/Sheets2DuckDB/Output/Parquet"

Autorizar conexão com o Google Sheets

In [35]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Importar planilha do Google Sheets

Ingerir dados da planilha do Google Sheets através da URL

In [46]:
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1Sc1vMlY094ApcBUTxlg11OpijETAAt08hj2wIUXzmMs/edit'
spreadsheet = gc.open_by_url(spreadsheet_url)

Importar dados da primeira aba da planilha


In [47]:
worksheet = spreadsheet.get_worksheet(0)

# Criar Dataframe e transformar dados

Criar DataFrame da planilha e obter todos os dados página

In [ ]:
df = pd.DataFrame(worksheet.get_all_records())
df.head()

# Outros

In [ ]:
# Salvar o DataFrame como arquivos Parquet
parquet_file = 'caminho/para/arquivo.parquet'
table = pa.Table.from_pandas(df)
pq.write_table(table, parquet_file)

# Autenticar na API do Google Drive
drive_service = build('drive', 'v3', credentials=credentials)
folder_id = 'ID_DA_PASTA_NO_GOOGLE_DRIVE'

# Upload do arquivo Parquet para a pasta do Google Drive
media = MediaFileUpload(parquet_file, mimetype='application/parquet')
file_metadata = {'name': 'dados.parquet', 'parents': [folder_id]}
file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
print('Arquivo Parquet enviado para o Google Drive com o ID:', file.get('id'))